In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install "git+https://github.com/albumentations-team/albumentations.git"

In [2]:
!pip install timm

You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
import sys
sys.path.append("utils")
sys.path.append("SwinT_detectron2")

In [2]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer
from albumentations import *
import torch
import os
from detectron2.data import detection_utils
from utils.aug import MyMapper
from utils.add_swint_config import add_swint_config
from detectron2.solver.build import *
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import warnings
import swint
warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
class Config:
  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]
  anchor_generators_sizes = [[8], [16], [32], [64],[128]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]
  model_config = "SwinT_detectron2/configs/SwinT/mask_rcnn_swint_T_FPN_3x.yaml"
  model_weights = "swin_L_384_22k.pth"

In [4]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=1) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
 
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    @classmethod
    def build_optimizer(cls, cfg, model):
        params = get_default_optimizer_params(
            model,
            base_lr=cfg.SOLVER.BASE_LR,
            weight_decay=cfg.SOLVER.WEIGHT_DECAY,
            weight_decay_norm=cfg.SOLVER.WEIGHT_DECAY_NORM,
            bias_lr_factor=cfg.SOLVER.BIAS_LR_FACTOR,
            weight_decay_bias=cfg.SOLVER.WEIGHT_DECAY_BIAS,
        )

        def maybe_add_full_model_gradient_clipping(optim):  # optim: the optimizer class
            # detectron2 doesn't have full model gradient clipping now
            clip_norm_val = cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE
            enable = (
                cfg.SOLVER.CLIP_GRADIENTS.ENABLED
                and cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE == "full_model"
                and clip_norm_val > 0.0
            )

            class FullModelGradientClippingOptimizer(optim):
                def step(self, closure=None):
                    all_params = itertools.chain(*[x["params"] for x in self.param_groups])
                    torch.nn.utils.clip_grad_norm_(all_params, clip_norm_val)
                    super().step(closure=closure)

            return FullModelGradientClippingOptimizer if enable else optim

        optimizer_type = cfg.SOLVER.OPTIMIZER
        if optimizer_type == "SGD":
            optimizer = torch.optim.SGD(
                params, cfg.SOLVER.BASE_LR, momentum=cfg.SOLVER.MOMENTUM,
                nesterov=cfg.SOLVER.NESTEROV,
                weight_decay=cfg.SOLVER.WEIGHT_DECAY,
            )
        elif optimizer_type == "AdamW":
              optimizer = torch.optim.AdamW(
                params, cfg.SOLVER.BASE_LR, betas=(0.9, 0.999),
                weight_decay=cfg.SOLVER.WEIGHT_DECAY,
            )
        else:
            raise NotImplementedError(f"no optimizer type {optimizer_type}")
        return optimizer

     # @classmethod
    # def build_train_loader(cls, cfg, sampler=None):
    #     return build_detection_train_loader(
    #         cfg, mapper=MyMapper(cfg), sampler=sampler
    #     )

    # def build_hooks(self):
    #   # copy of cfg
    #   cfg = self.cfg.clone()

    #   # build the original model hooks
    #   hooks = super().build_hooks()

    #   # add the best checkpointer hook
    #   hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
    #                                     DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
    #                                     "MaP IoU",
    #                                     "max",
    #                                     ))
      
def setup():
  cfg = get_cfg()
  add_swint_config(cfg)
  cfg.merge_from_file(Config.model_config)
  #cfg.MODEL.WEIGHTS = Config.model_weights
  #cfg.merge_from_list(args.opts)
  #cfg.freeze()
  #default_setup(cfg, args)
  return cfg


In [5]:
def run():
  dataDir = "data"
  DatasetCatalog.clear()
  MetadataCatalog.clear()
  register_coco_instances(f'sartorius_train',{}, 'input/all/annotations_train.json'.format(fold), dataDir)
  register_coco_instances(f'sartorius_val',{},'input/all/annotations_val.json'.format(fold), dataDir)

  #cfg = get_cfg()
  #cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"))
  cfg = setup()
   # RCNN
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.INPUT.MIN_SIZE_TRAIN = (1000,1100,1200,1300)
  cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
  cfg.INPUT.MIN_SIZE_TEST = 1760
  cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
  cfg.SOLVER.CLIP_GRADIENTS = False

  cfg.SOLVER.OPTIMIZER = "AdamW"
  cfg.MODEL.BACKBONE.FREEZE_AT = -1

  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000

#   cfg.MODEL.PIXEL_MEAN = [128,128,128]
#   cfg.MODEL.PIXEL_STD = [13.235,13.235,13.235]
  cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[16], [32], [64], [128],[256]]
  cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[0.5, 1.0, 2.0]]

  cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 3000
  cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 3000
  cfg.SOLVER.BASE_LR = 1e-3
  cfg.DATASETS.TRAIN = (f"sartorius_train",)
  cfg.DATASETS.TEST = (f"sartorius_val",)
  #cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
  cfg.SOLVER.MAX_ITER = 10000 #尝试20ep 用lr调整
  cfg.SOLVER.STEPS = (5000,7500)
  cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

  #cfg.MODEL.BACKBONE.FREEZE_AT = 2
  #cfg.MODEL.RESNETS.DEPTH = 101
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5","p6"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]

  # cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True] # on Res3,Res4,Res5
  # cfg.MODEL.RESNETS.DEFORM_MODULATED = True
  # cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 2
  # cfg.MODEL.RESNETS.NORM = "GN"
  # cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
  # cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 4
  # cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 1
  # cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"  
  # cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 7
  # cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True  
  # cfg.MODEL.ROI_MASK_HEAD.NUM_CONV = 8
  # cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
  # cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS = (0.5, 0.6, 0.7)
  

  cfg.OUTPUT_DIR = "finetuned/swinT/".format(fold)

  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [ ]:
# 2 的理论score最高
for fold in range(1,2):
  run()

[12/20 23:48:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(384, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(1536, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inp

Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.norm0.{bias, weight}
backbone.bottom_up.norm1.{bias, weight}
backbone.bottom_up.norm2.{bias, weight}
backbone.bottom_up.norm3.{bias, weight}
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.

[12/20 23:48:04 d2.engine.train_loop]: Starting training from iteration 0


[W LegacyTypeDispatch.h:74] Warning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (function operator())


[12/20 23:48:32 d2.utils.events]:  eta: 2:19:25  iter: 19  total_loss: 17.75  loss_cls: 1.284  loss_box_reg: 0.009031  loss_mask: 0.692  loss_rpn_cls: 0.6985  loss_rpn_loc: 15.1  time: 1.2530  data_time: 0.4826  lr: 1.9981e-05  max_mem: 35005M
[12/20 23:48:55 d2.utils.events]:  eta: 2:13:47  iter: 39  total_loss: 25.23  loss_cls: 1.005  loss_box_reg: 0.01412  loss_mask: 0.6737  loss_rpn_cls: 0.5922  loss_rpn_loc: 22.86  time: 1.2131  data_time: 0.4413  lr: 3.9961e-05  max_mem: 35005M
[12/20 23:49:13 d2.utils.events]:  eta: 2:10:09  iter: 59  total_loss: 15.87  loss_cls: 0.33  loss_box_reg: 0.06235  loss_mask: 0.5767  loss_rpn_cls: 0.4235  loss_rpn_loc: 14.36  time: 1.0947  data_time: 0.1284  lr: 5.9941e-05  max_mem: 35005M
[12/20 23:49:37 d2.utils.events]:  eta: 2:10:18  iter: 79  total_loss: 21.45  loss_cls: 0.5076  loss_box_reg: 0.5074  loss_mask: 0.5549  loss_rpn_cls: 0.328  loss_rpn_loc: 19.38  time: 1.1300  data_time: 0.4753  lr: 7.9921e-05  max_mem: 35005M
[12/20 23:50:08 d2.util